In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

np.set_printoptions(suppress=True, precision=3)

sns.set(style='ticks', palette='Set2')
sns.despine()

In [2]:
import utils as U
import opt_runs as OR
import opt_model as OM
import decorated_options as Deco

# Step 1: Create a SimOpts which will generate raw data

In [3]:
sim_opts_1_follower = OM.SimOpts(
    src_id=0,
    end_time=100,
    s=np.array([1]),
    q=1.0,
    other_sources=[('Hawkes', {'src_id': 1, 'seed': 1, 'l_0': 1.0, 'alpha': 1.0, 'beta': 5.0})],
    sink_ids=[1000],
    edge_list=[(0, 1000), (1, 1000)]
)

# Step 2: Simulate the walls along with different strategies for our broadcaster

In [4]:
%%time
seed = 1
opt_mgr = sim_opts_1_follower.create_manager_with_opt(seed)
opt_mgr.run_dynamic()
opt_df = opt_mgr.state.get_dataframe()
num_opt_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_opt = {
    'type': 'Opt',
    'seed': seed,
    'capacity': num_opt_tweets,
    'q': sim_opts_1_follower.q
}
OR.add_perf(perf_opt, opt_df, sim_opts_1_follower)

CPU times: user 56.7 ms, sys: 0 ns, total: 56.7 ms
Wall time: 54.5 ms


In [5]:
%%time
seed = 9
poisson_mgr = sim_opts_1_follower.create_manager_with_poisson(seed, capacity=num_opt_tweets)
poisson_mgr.run_dynamic()
poisson_df = poisson_mgr.state.get_dataframe()
num_poisson_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_poisson = {
    'type': 'Poisson',
    'seed': seed,
    'capacity': num_poisson_tweets,
    'q': sim_opts_1_follower.q
}
OR.add_perf(perf_poisson, poisson_df, sim_opts_1_follower)

CPU times: user 70 ms, sys: 0 ns, total: 70 ms
Wall time: 68.8 ms


# Step 3: Calculate the performance

In [6]:
print('num_opt_tweets = {}, num_poisson_tweets = {}'
      .format(U.num_tweets_of(opt_df, 0), U.num_tweets_of(poisson_df, 0)))

num_opt_tweets = 78.0, num_poisson_tweets = 78.0


In [7]:
print('avg_rank_opt = {}, avg_rank_poisson = {}'
      .format(U.average_rank(opt_df, sim_opts=sim_opts_1_follower), 
              U.average_rank(poisson_df, sim_opts=sim_opts_1_follower)))

avg_rank_opt = 66.8116376874034, avg_rank_poisson = 201.3461740580856


In [8]:
perf_opt

{'avg_rank': 66.8116376874034,
 'capacity': 78.0,
 'num_events': 78,
 'q': 1.0,
 'r_2': 138.15727122691501,
 'seed': 1,
 'top_1': 56.517519051405124,
 'type': 'Opt',
 'world_events': 154}

In [9]:
print('top_1_opt = {}, top_1_poisson = {}'
      .format(perf_opt['top_1'], perf_poisson['top_1']))

top_1_opt = 56.517519051405124, top_1_poisson = 39.68728199394901


In [10]:
U.time_in_top_k(opt_df, src_id=1, K=1, sim_opts=sim_opts_1_follower)

43.48248094859487